# Sync Bullet Overflow

Sync vault note tracking overflow tasks in bullet list.

- https://markdown-it-py.readthedocs.io/en/latest/using.html#markdown-renderer
- https://mdformat.readthedocs.io/en/stable/users/installation_and_usage.html

In [ ]:
from datetime import datetime
from pathlib import Path
from subprocess import run
from urllib.parse import quote, unquote

from mdformat.renderer import MDRenderer
from mdformat_frontmatter import plugin
from more_itertools import one
from pandas import DataFrame, read_csv

from notes import bullet
from notes.markdown import MD
from notes.times import current_tz

DEBUG = False

has_id = r"title.str.contains('🆔', na=False)"
to_do = r"list_title == 'To-do'"
overflow = r"list_title == 'Overflow'"
done = r"date_complete.notna()"
linked = r"content.str.contains('\[note\]', na=False)"


def format_datetime(date: datetime) -> str:
    return date.astimezone(current_tz).isoformat(timespec="seconds").replace(":", "")


def format_date(date: datetime) -> str:
    return date.astimezone(current_tz).date().isoformat()


def qall(*args: str) -> str:
    return " & ".join(args)


def qnot(*args: str) -> str:
    return qall(*[f"~{arg}" for arg in args])


def format_link(link: str) -> str:
    link = (
        unquote(encoding="utf-8", string=link)
        .strip(")")
        .replace("&heading=", "#")
        .rsplit(adv_uri_fp if (adv_uri_fp := "&filepath=") in link else "&file=")[-1]
    )
    if "#" not in link:
        return link
    link, heading = link.rsplit("#")
    return f"{link}#{quote(encoding='utf-8', string=heading)}"


def format_linked(df: DataFrame) -> str:
    return "\n".join(
        f"- [{r['title']}]({format_link(r['content'])})" for r in df.to_dict("records")
    )


if DEBUG:
    run(args=["uv", "run", "-m", "notes.bullet"], check=True)

today = format_date(datetime.now())
bullet_export = (
    bullet.PATH
    if DEBUG
    else Path(f"data/local/vaults/personal/_data/bullet_export_{today}.csv")
)
if not bullet_export.exists():
    raise ValueError(
        f"Bullet export file {bullet_export.stem} not found. Did you forget to export it?"
    )
bullet_export_tracking_note = Path(
    "D:/code/mine/notes/data/local/vaults/personal/_logs/tasks/current.md"
)
bullets = read_csv(bullet_export, engine="pyarrow").query("type == 'task'")
tokens = MD.parse(bullet_export_tracking_note.read_text(encoding="utf-8"))

bullets_preview_ = bullets.dropna(axis="columns", how="all")
tokens_preview_ = DataFrame(tokens)


text = "\n\n".join([
    MDRenderer().render(
        tokens=tokens[
            : tokens.index(
                one(t for t in tokens if t.type == "heading_open" and t.tag == "h1")
            )
        ],
        options={"parser_extension": [plugin]},
        env={},
        finalize=False,
    ),
    f"# Current ({today})",
    "Synchronized with bullet list.",
    "## Linked, no ID",
    bullets.query(qall(to_do, linked, qnot(has_id)))
    .sort_values(by="date_created", ascending=False)
    .pipe(format_linked),
    "## Linked",
    bullets.query(qall(to_do, linked, has_id, qnot(done)))
    .sort_values(
        by="title", key=lambda ser: ser.str.split("🆔 ").str[-1], ascending=False
    )
    .pipe(format_linked),
    "## Do",
    bullets.query(qall(to_do, qnot(done, has_id)))
    .sort_values(by="date_created", ascending=False)
    .pipe(
        lambda df: "\n".join(
            f"- [ ] {r['title']} 🆔 {format_datetime(r['date_created'])}"
            for r in df.to_dict("records")
        )
    ),
    "## Overflow",
    bullets.query(qall(overflow, qnot(has_id)))
    .sort_values(by="date_created", ascending=False)
    .pipe(
        lambda df: "\n".join(
            f"- [ ] {r['title']} 🆔 {format_datetime(r['date_created'])}"
            for r in df.to_dict("records")
        )
    ),
    "## Linked, complete",
    bullets.query(qall(to_do, linked, has_id, done))
    .sort_values(
        by="title", key=lambda ser: ser.str.split("🆔 ").str[-1], ascending=False
    )
    .pipe(format_linked),
    "## Complete",
    bullets.query(qall(to_do, done, qnot(has_id)))
    .sort_values(by="date_complete", ascending=False)
    .pipe(
        lambda df: "\n".join(
            f"- [x] {r['title']}"
            f" 🆔 {format_datetime(r['date_created'])}"
            f" ✅ {format_date(r['date_complete'])}"
            for r in df.to_dict("records")
        )
    ),
])

bullet_export_tracking_note.write_text(encoding="utf-8", data=text)
print(text)